In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1317


In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0
n = 500

data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [5]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  14.32 MiB
  allocs estimate:  220096
  --------------
  minimum time:     17.323 ms (0.00% GC)
  median time:      22.672 ms (0.00% GC)
  mean time:        24.079 ms (12.55% GC)
  maximum time:     42.552 ms (36.11% GC)
  --------------
  samples:          208
  evals/sample:     1

In [6]:
inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=1.0342381334123159, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=2.034238133412316, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=3.0342381334123156, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=4.034238133412314, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.034238133412315, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=6.034238133412317, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=7.034238133412316, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=8.034238133412314, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=9.034238133412314, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.034238133412314, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=11.034238133412313, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float6